### AccelerateAI - Python for Data Science
Introduction to Python Language (Python 3)

In this notebook we will cover the following:
 - Processing JSON and XLS 
 - Web Scraping
 - Relational Database
 - Data Pipeline

#### 1.1 JSON : Javascript Object Notation 
- JSON is a text format that is often used to exchange data on the web.

In [ ]:
# python package json - can be used to work with JSON data.
import json

In [ ]:
json_data = '''
[    {"name" : "Alan",
    "phone" : "+1 734 303 4456",
    "email" : "hidden"
    },
    {"name" : "Chuck",
    "phone" : "+1 444 503 4456",
    "email" : " "
    }
]
'''

In [ ]:
py_data = json.loads(json_data)

In [ ]:
py_data

In [ ]:
for item in py_data:
    print("Name:", item["name"])
    print("Phone:", item["phone"])
    print("Email:", item["email"])


- json.loads() is a Python list which we traverse with a for loop, 
- Each item within that list is a Python dictionary. 
- Once the JSONhas been parsed, we can use the Python index operator to extract the various bits of data for each user.

In [ ]:
emp = '''[
      {
        "id": "MI5",
        "firstName": "Tom",
        "lastName": "Cruise"
      },
      {
        "id": "N1",
        "firstName": "Maria",
        "lastName": "Sharapova"
      },
      {
        "id": "007",
        "firstName": "James",
        "lastName": "Bond"
      }
    ]
'''

In [ ]:
#reading JSON with pandas
import pandas as pd

df = pd.read_json(emp)
print(df)

In [ ]:
import requests
from pandas.io.json import json_normalize
import pandas as pd

url = "https://api.exchangerate.host/latest"
df = pd.read_json(url)

In [ ]:
df 

In [ ]:
#json_normalize() for nested dict's

data = [
    {
        "id": 1,
        "name": "Cole Volk",
        "fitness": {"height": 130, "weight": 60},
    },
    {"name": "Mark Reg", "fitness": {"height": 130, "weight": 60}},
    {
        "id": 2,
        "name": "Faye Raker",
        "fitness": {"height": 130, "weight": 60},
    },
]

pd.json_normalize(data, max_level=1)

#### 1.2 xml - eXtensible Markup Language
- Similar to HTML but more structured 
- designed to send and receive data between clients and servers
- best suited for exchanging document-style data

In [ ]:
data = '''
<person>
    <name>Alan</name>
    <phone type="intl"> +1 734 303 4456 </phone>
    <email hide="yes"/>                                        #empty depicted  by />
</person> 
'''

In [ ]:
#The xml.etree.ElementTree module implements API for parsing and creating XML data. 
import xml.etree.ElementTree as ET                         # ElementTree represents the whole XML document as a tree

In [ ]:
root = ET.fromstring(data)                                 # parse from a string 

In [ ]:
for child in root:
    print(child.tag, child.attrib, child.text) 

In [ ]:
tree = ET.parse('country.xml.txt')
root = tree.getroot()

In [ ]:
for child in root:
    print(child.tag, child.attrib, child.text)

In [ ]:
for x in root.findall('country'):
    rank = x.find('rank').text
    year = x.find('year').text
    gdp  = x.find('gdppc').text
    print(x.attrib.values(), rank, year, gdp)

#### 2. Web Scraping

##### Process of Webscraping: 
1) Request the content (source code) of a specific URL from the server <br>
2) Download the content that is returned from server <br>
3) Identify the elements of the page that are part of the table <br>
4) Extract and reformat those elements into a dataset for analysis

##### Python Libraries for WebScraping
- Beautiful Soup is a Python library for pulling data out of HTML and XML files. It provides methods for navigating, searching, and modifying a parse tree.
- Scrapy is a fast high-level web crawling & scraping framework for Python
- Selenium Python is an open-source web-based automation tool which provides a simple API to write functional or acceptance tests using Selenium WebDriver

In [160]:
#import 
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [161]:
#We use requests to fetch the web document
url = 'https://www.flipkart.com/laptops/dell~brand/pr?sid=6bo,b5g'
data = requests.get(url)

In [163]:
#print(data.text)

In [164]:
soup = BeautifulSoup(data.text, "html.parser")

In [165]:
print(soup.prettify())

<!DOCTYPE doctype html>
<html lang="en">
 <head>
  <link href="https://rukminim1.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/www/linchpin/fk-cp-zion/css/app_modules.chunk.94b5e7.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/www/linchpin/fk-cp-zion/css/app.chunk.9adf7d.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="102988293558" property="fb:page_id"/>
  <meta content="658873552,624500995,100000233612389" property="fb:admins"/>
  <meta content="noodp" name="robots"/>
  <link href="https://static-assets-web.flixcart.com/www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/>
  <link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/>
  <meta content="website" property="og:type"/>
  <meta content="Flipkart.com" name="og_site_name" property="og:site_name"/>
  <link

In [166]:
items = soup.select('a._1fQZEK')

In [167]:
item = items[0]
#product_name = item.select('._4rR01T')[0].get_text()
product_name = item.find('div', attrs={'class':'_4rR01T'}).contents[0]
print(product_name)

DELL Inspiron Core i3 11th Gen - (8 GB/1 TB HDD/256 GB SSD/Windows 11 Home) Inspiron 3511 Thin and Lig...


In [173]:
discounted_price= item.find('div', attrs={'class':'_30jeq3 _1_WHN1'}).contents[0]
print(discounted_price)

₹45,390


In [171]:
original_price= item.find('div', attrs={'class':'_3I9_wc _27UcVY'}).contents[2]
print(original_price)

60,371


In [174]:
user_rating = item.find('div', attrs={'class':'_3LWZlK'}).contents[0]
print(user_rating)

4.4


##### Let's do all the above in a loop ! 

In [ ]:
names = [] #List to store name of the product
oprices = []  #List to store MRP of the product
dprices = []  #List to store discounted price of the product
ratings = []

In [175]:
for item in soup.findAll('a', href=True, attrs={'class':'_1fQZEK'}):
    name = item.find('div', attrs={'class':'_4rR01T'}).contents[0]
    dprice= item.find('div', attrs={'class':'_30jeq3 _1_WHN1'}).contents[0]
    oprice= item.find('div', attrs={'class':'_3I9_wc _27UcVY'}).contents[2]
    rating = item.find('div', attrs={'class':'_3LWZlK'})
    
    names.append(name)
    dprices.append(dprice)   
    oprices.append(oprice)
    ratings.append(rating)

In [176]:
product_catalog_df = pd.DataFrame({"Product":names, 
                                   "Original Price":oprices,
                                   "Discounted Price":dprices, 
                                   "User Rating:":ratings})

In [177]:
product_catalog_df.head()

,Product,Original Price,Discounted Price,User Rating:
0,DELL Inspiron Core i3 11th Gen - (8 GB/1 TB HD...,"60,371","₹45,390",4.4
1,DELL Inspiron Core i3 11th Gen - (8 GB/1 TB HD...,"57,479","₹44,490",4.3
2,DELL Inspiron Ryzen 3 Dual Core 3250U - (8 GB/...,"47,695","₹37,390",4.2
3,DELL Inspiron Core i3 11th Gen - (8 GB/512 GB ...,"59,712","₹48,990",4.3
4,DELL Vostro Core i3 10th Gen - (8 GB/256 GB SS...,"43,312","₹41,490",4.3


##### The same can be acheived using Scrappy - though it can be a bit more work.

- Scrapy creates a spider(bot) and can crawl multiple web pages to extract information and store it 

#### 3. Relational Database

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

if(conn):
    print("\nDatabase created and connected to SQLite.")
    sqlite_select_Query = "select sqlite_version();"
    conn.execute(sqlite_select_Query)
    print("\nSQLite Database Version is: ", record)

In [ ]:
cur.execute('CREATE TABLE IF NOT EXISTS Tracks (title TEXT, plays INTEGER)')

In [ ]:
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', ('Thunderstruck', 180))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', ('Hotel California', 240))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', ('Believer', 190))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', ('Country Roads', 175))

conn.commit()

In [ ]:
cur.execute('SELECT * FROM Tracks')
print('Tracks:')
for row in cur:
    print(row)

In [ ]:
cur.execute('SELECT * FROM Tracks WHERE plays > 200')
for row in cur:
    print(row)

In [ ]:
# Retreive data using pandas 
df = pd.read_sql_query("SELECT * from Tracks", conn)
print(df.head())

In [ ]:
#writing data to SQL database using pandas 

newsong_df = pd.DataFrame({"title":["My heart will go on"], 
                           "plays": [360]})

newsong_df.to_sql("Tracks", conn, if_exists="append", index=False)               #index=False is needed !

In [ ]:
cur.execute('SELECT * FROM Tracks WHERE plays > 300')
for row in cur:
    print(row)

In [ ]:
cur.execute("DROP TABLE Tracks")
conn.commit()

In [ ]:
cur.close()

In [ ]:
# Do these in a try-catch block 
try:
    sqlite_Connection = sqlite3.connect('temp.db')
    conn = sqlite_Connection.cursor()
    record = conn.fetchall()
    # Do something
    conn.close()
except sqlite3.Error as error:
    print("\nConnection error:", error)
    #handle error 
finally:
    if (sqlite_Connection):
        sqlite_Connection.close()
        print("\nConnection closed.")